# Project Part 1 - Common Analysis

**Author: Logan O'Brien** 

## Step 1 Continued: Create the Fire Smoke Estimates

Once again we begin by importing the necessary packages

In [1]:
import os, json, time
from pyproj import Transformer, Geod

#importing [4]
from wildfire.Reader import Reader as WFReader

import numpy as np
import requests
from tqdm import tqdm
import pandas as pd
from datetime import datetime

**Note, I heavily rely upon Dr. McDonald's "wildfire_geo_proximity_example.ipynb" to learn how the wildfire data is structured as well as how to work with it [3]. In the subsequent steps, I will call out when I use code from that notebook.**

Now, at this point we have already filtered the wildfire data to the fires we care about in another notebook. So let's load that filtered data and continue. 

We begin by loading the data, where I employ modified code from two cells in [3].

In [2]:
# I modified this file path to make it point to where I stored the file
# DATA_FILENAME = './wildfire/Wildfire_short_sample.json'
DATA_FILENAME = './data/relevant_fires.json'

#
#    This bit of code opens a new wildfire reader
#
print(f"Attempting to open '{DATA_FILENAME}' with wildfire.Reader() object")
wfreader = WFReader(DATA_FILENAME)

Attempting to open './data/relevant_fires.json' with wildfire.Reader() object


Now, let's use the Professor's Reader program in his wildfire module to load the data [4]. I employ a cell from [3] and modify some comments. 

In [3]:
#    This code loads file, or a small amount of the complete dataset
MAX_FEATURE_LOAD = 500
feature_list = list()
feature_count = 0
# A rewind() on the reader object makes sure we're at the start of the feature list
# This way, we can execute this cell multiple times and get the same result 
wfreader.rewind()
# Now, read through each of the features, saving them as dictionaries into a list
feature = wfreader.next()
while feature:
    feature_list.append(feature)
    feature_count += 1
    # if we're loading a lot of features, print progress
    if (feature_count % 100) == 0:
        print(f"Loaded {feature_count} features")
    # loaded the max we're allowed then break
#     if feature_count >= MAX_FEATURE_LOAD:
#         break
    feature = wfreader.next()
#
#    Print the number of items (features) we think we loaded
print(f"Loaded a total of {feature_count} features")
#
#    Just a validation check - did all the items we loaded get into the list?
print(f"Variable 'feature_list' contains {len(feature_list)} features")

Loaded 100 features
Loaded 200 features
Loaded 300 features
Loaded 400 features
Loaded 500 features
Loaded 600 features
Loaded 700 features
Loaded 800 features
Loaded 900 features
Loaded 1000 features
Loaded 1100 features
Loaded 1200 features
Loaded 1300 features
Loaded 1400 features
Loaded 1500 features
Loaded 1600 features
Loaded 1700 features
Loaded 1800 features
Loaded 1900 features
Loaded 2000 features
Loaded 2100 features
Loaded 2200 features
Loaded 2300 features
Loaded 2400 features
Loaded 2500 features
Loaded 2600 features
Loaded 2700 features
Loaded 2800 features
Loaded 2900 features
Loaded 3000 features
Loaded 3100 features
Loaded 3200 features
Loaded 3300 features
Loaded 3400 features
Loaded 3500 features
Loaded 3600 features
Loaded 3700 features
Loaded 3800 features
Loaded 3900 features
Loaded 4000 features
Loaded 4100 features
Loaded 4200 features
Loaded 4300 features
Loaded 4400 features
Loaded 4500 features
Loaded 4600 features
Loaded 4700 features
Loaded 4800 features
L

Loaded 37900 features
Loaded 38000 features
Loaded 38100 features
Loaded 38200 features
Loaded 38300 features
Loaded 38400 features
Loaded 38500 features
Loaded 38600 features
Loaded 38700 features
Loaded 38800 features
Loaded 38900 features
Loaded 39000 features
Loaded 39100 features
Loaded 39200 features
Loaded 39300 features
Loaded 39400 features
Loaded 39500 features
Loaded 39600 features
Loaded 39700 features
Loaded 39800 features
Loaded 39900 features
Loaded 40000 features
Loaded 40100 features
Loaded 40200 features
Loaded 40300 features
Loaded 40400 features
Loaded 40500 features
Loaded 40600 features
Loaded 40700 features
Loaded 40800 features
Loaded 40900 features
Loaded 41000 features
Loaded 41100 features
Loaded 41200 features
Loaded 41300 features
Loaded 41400 features
Loaded 41500 features
Loaded 41600 features
Loaded 41700 features
Loaded 41800 features
Loaded 41900 features
Loaded 42000 features
Loaded 42100 features
Loaded 42200 features
Loaded 42300 features
Loaded 424

Loaded 75700 features
Loaded 75800 features
Loaded 75900 features
Loaded 76000 features
Loaded 76100 features
Loaded 76200 features
Loaded 76300 features
Loaded 76400 features
Loaded a total of 76407 features
Variable 'feature_list' contains 76407 features


## Create an Estimate for the Smoke Severity

The next task is to design a smoke estimate and apply it to my fires.

**Description of my Smoke Estimate:**

Our next task is to create a smoke estimate.
First, I consider what will I allow for my range of values. According to airnow.gov, the AQI index ranges from 0 to 500, where 0 is the cleanest air and 500 is the worst [10]. Now, because the assignment requires me to compare my smoke estimate to the AQI information from the EPA, I would like to try to encourage a similar range for most of my estimates. 

Next, I consider what factors I want to be incorporated into my smoke severity estimate. The assignment instructions provide several helpful ideas saying "It seems reasonable that a large fire, that burns a large number of acres, and that is close to a city would put more smoke into a city than a small fire that is much further away." After giving it some thought, I will base my estimate off 1). each fire's size, measured in the number of acres burned, and 2). its distance from Richland as calculated earlier above.

Let's think about each of these factors, starting with acres burned. Now, it makes sense that my smoke estimate should be increase for larger fires and decrease for smaller fires, all else being equal. Additionally, a fire closer to Richland should result in poorer smoke conditions than a fire far away. Taken together, these two factors are at odds with each other, and so I decided to take their ratio for my smoke estimate - yielding: ***acres_burned / distance_from_city***.

There are several more changes to make to this estimate to improve it a bit. First, we can convert the acres burned to square miles burned so that the numerator and denominator have the same units. As an aside, the resulting units are now 1/mi, and to be honest I am not sure how to think about a smoke estimator with a unit attached to it, so for the sake of this assignment, I will not concern myself with the estimate's units. 

While designing my estimator, I revisited the course lecture notes from 10-30-23 and looked at the professor's findings from his exporatory data analysis. According to lecture, about 94% of all fires are between 0 and 5000 acres (approx. 7.8 square miles), but the largest was 1.4M acres (approx. 2,100 square miles). In contrast, the distances I will consider will be no greater than 1,250 miles from Richland. Because the majority of these areas burned are small relative to the distances, I added a scaling factor to the numerator so it will be weighted more evenly, by multiplying by 100. 

I also wanted to reduce the frequency of more extreme values in my smoke estimates, and so I enforce a heuristic that the minimum distance a fire can be from the city is 1 mile, and that any fire closer than that is treated by the estimate as 1 mile. Simillarly, I enforce a minimum size of burned area (0.5 square miles). Lastly, I want to ensure my estimate has a minimum value of 1, so I take the argmax of the expression and thus my final estimate is:
$$argmax(\frac{\text{(square_mi_burned * 100)}}{\text{distance_from_city}}, 1)$$

Below is the function I defined to calculate my smoke estimate.

In [4]:
#distance_from_city should be provided in miles
def calc_smoke_severity_est(acres_burned, distance_from_city):
    SCALING_FACTOR = 100
    
    #convert acres to square miles. I got formula from Google [10]
    square_mi_burned = acres_burned / 640
#     print("acres_burned {}".format(acres_burned))
#     print("square_mi_burned {}".format(square_mi_burned))
    
    #to simplify the estimate, set 1 as the minimum
    #distance and 0.5 as the minimum square miles burned
#     print("distance from Richland {}".format(distance_from_city))
    if distance_from_city < 1:
        distance_from_city = 1
    
    if square_mi_burned < 0.5:
        square_mi_burned = 0.5
    
    # I learned how to do argmax() in python from [12] and [13]
    value = (square_mi_burned * SCALING_FACTOR) / distance_from_city
    options = [value, 1]
    index = np.argmax([(square_mi_burned * SCALING_FACTOR) / distance_from_city, 1])
    result = options[index]
#     print("Result: {}".format(result))
#     print('')
    return result

Now, let's apply the function to my list of relevant fires and store the results in a dataframe. First we convert the list of fires to a dataframe.

In [5]:
# Consulted for how to create the dataframe:
# https://www.geeksforgeeks.org/create-a-pandas-dataframe-from-list-of-dicts/#
feature_list
df_fires = pd.DataFrame.from_records(feature_list)
df_fires

,OBJECTID,USGS_Assigned_ID,Assigned_Fire_Type,Fire_Year,Fire_Polygon_Tier,Fire_Attribute_Tiers,GIS_Acres,GIS_Hectares,Source_Datasets,Listed_Fire_Types,...,Wildfire_Notice,Prescribed_Burn_Notice,Wildfire_and_Rx_Flag,Overlap_Within_1_or_2_Flag,Circleness_Scale,Circle_Flag,Exclude_From_Summary_Rasters,Shape_Length,Shape_Area,distance_to_richland
0,14299,14299,Wildfire,1963,1,"1 (1), 3 (3)",40992.458271,16589.059302,Comb_National_NIFC_Interagency_Fire_Perimeter_...,"Wildfire (1), Likely Wildfire (3)",...,Wildfire mapping prior to 1984 was inconsisten...,Prescribed fire data in this dataset represent...,None,None,0.385355,NaN,No,73550.428118,1.658906e+08,189.757596
1,14300,14300,Wildfire,1963,1,"1 (1), 3 (3)",25757.090203,10423.524591,Comb_National_NIFC_Interagency_Fire_Perimeter_...,"Wildfire (2), Likely Wildfire (2)",...,Wildfire mapping prior to 1984 was inconsisten...,Prescribed fire data in this dataset represent...,None,None,0.364815,NaN,No,59920.576713,1.042352e+08,163.213533
2,14301,14301,Wildfire,1963,1,"1 (5), 3 (15), 5 (1)",45527.210986,18424.208617,Comb_National_NIFC_Interagency_Fire_Perimeter_...,"Wildfire (6), Likely Wildfire (15)",...,Wildfire mapping prior to 1984 was inconsisten...,Prescribed fire data in this dataset represent...,None,None,0.320927,NaN,No,84936.827810,1.842421e+08,190.823448
3,14302,14302,Wildfire,1963,1,"1 (1), 3 (3), 5 (1)",10395.010334,4206.711433,Comb_National_NIFC_Interagency_Fire_Perimeter_...,"Wildfire (2), Likely Wildfire (3)",...,Wildfire mapping prior to 1984 was inconsisten...,Prescribed fire data in this dataset represent...,None,None,0.428936,NaN,No,35105.903602,4.206711e+07,273.615572
4,14303,14303,Wildfire,1963,1,"1 (1), 3 (3)",9983.605738,4040.221900,Comb_National_NIFC_Interagency_Fire_Perimeter_...,"Wildfire (1), Likely Wildfire (3)",...,Wildfire mapping prior to 1984 was inconsisten...,Prescribed fire data in this dataset represent...,None,None,0.703178,NaN,No,26870.456126,4.040222e+07,205.770574
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76402,135057,135057,Prescribed Fire,2020,8,8 (3),16.412148,6.641761,Comb_National_Rx_Only_BLM_VTRT_Prescribed_Fire...,Prescribed Fire (3),...,Wildfire mapping prior to 1984 was inconsisten...,Prescribed fire data in this dataset represent...,None,"Caution, this Prescribed Fire in 2020 overlaps...",0.177425,NaN,No,2168.900740,6.641761e+04,248.914999
76403,135058,135058,Prescribed Fire,2020,8,8 (1),7.050837,2.853373,Comb_National_Rx_Only_BLM_VTRT_Prescribed_Fire...,Prescribed Fire (1),...,Wildfire mapping prior to 1984 was inconsisten...,Prescribed fire data in this dataset represent...,None,"Caution, this Prescribed Fire in 2020 overlaps...",0.374368,NaN,No,978.666221,2.853373e+04,167.983467
76404,135059,135059,Prescribed Fire,2020,8,8 (4),9.342668,3.780843,Comb_National_Rx_Only_BLM_VTRT_Prescribed_Fire...,Prescribed Fire (4),...,Wildfire mapping prior to 1984 was inconsisten...,Prescribed fire data in this dataset represent...,None,"Caution, this Prescribed Fire in 2020 overlaps...",0.123888,NaN,No,1958.326660,3.780843e+04,168.822089
76405,135060,135060,Prescribed Fire,2020,8,8 (1),0.996962,0.403456,Comb_National_Rx_Only_BLM_VTRT_Prescribed_Fire...,Prescribed Fire (1),...,Wildfire mapping prior to 1984 was inconsisten...,Prescribed fire data in this dataset represent...,None,None,0.993809,1.0,No,225.866452,4.034562e+03,655.423767


Next we apply our smoke estimate and store it.

In [6]:
# I tried to vectorize this but couldn't figure it out: 
# https://tryolabs.com/blog/2023/02/08/top-5-tips-to-make-your-pandas-code-absurdly-fast

#https://stackoverflow.com/questions/16476924/how-to-iterate-over-rows-in-a-dataframe-in-pandas
# recommends using apply, but I used the webpage to learn how to use iterrows anyway.

# df_fires['my_smoke_estimate'] = calc_smoke_severity_est(df_fires['GIS_Acres'],
#                                                         df_fires['distance_to_richland'])
smoke_est_list = []

for index, row in df_fires.iterrows():
    smoke_est = calc_smoke_severity_est(row['GIS_Acres'], row['distance_to_richland'])
    smoke_est_list.append(smoke_est)
    
#add column to dataframe
df_fires['my_smoke_estimate'] = smoke_est_list
df_fires

,OBJECTID,USGS_Assigned_ID,Assigned_Fire_Type,Fire_Year,Fire_Polygon_Tier,Fire_Attribute_Tiers,GIS_Acres,GIS_Hectares,Source_Datasets,Listed_Fire_Types,...,Prescribed_Burn_Notice,Wildfire_and_Rx_Flag,Overlap_Within_1_or_2_Flag,Circleness_Scale,Circle_Flag,Exclude_From_Summary_Rasters,Shape_Length,Shape_Area,distance_to_richland,my_smoke_estimate
0,14299,14299,Wildfire,1963,1,"1 (1), 3 (3)",40992.458271,16589.059302,Comb_National_NIFC_Interagency_Fire_Perimeter_...,"Wildfire (1), Likely Wildfire (3)",...,Prescribed fire data in this dataset represent...,None,None,0.385355,NaN,No,73550.428118,1.658906e+08,189.757596,33.753967
1,14300,14300,Wildfire,1963,1,"1 (1), 3 (3)",25757.090203,10423.524591,Comb_National_NIFC_Interagency_Fire_Perimeter_...,"Wildfire (2), Likely Wildfire (2)",...,Prescribed fire data in this dataset represent...,None,None,0.364815,NaN,No,59920.576713,1.042352e+08,163.213533,24.658160
2,14301,14301,Wildfire,1963,1,"1 (5), 3 (15), 5 (1)",45527.210986,18424.208617,Comb_National_NIFC_Interagency_Fire_Perimeter_...,"Wildfire (6), Likely Wildfire (15)",...,Prescribed fire data in this dataset represent...,None,None,0.320927,NaN,No,84936.827810,1.842421e+08,190.823448,37.278578
3,14302,14302,Wildfire,1963,1,"1 (1), 3 (3), 5 (1)",10395.010334,4206.711433,Comb_National_NIFC_Interagency_Fire_Perimeter_...,"Wildfire (2), Likely Wildfire (3)",...,Prescribed fire data in this dataset represent...,None,None,0.428936,NaN,No,35105.903602,4.206711e+07,273.615572,5.936140
4,14303,14303,Wildfire,1963,1,"1 (1), 3 (3)",9983.605738,4040.221900,Comb_National_NIFC_Interagency_Fire_Perimeter_...,"Wildfire (1), Likely Wildfire (3)",...,Prescribed fire data in this dataset represent...,None,None,0.703178,NaN,No,26870.456126,4.040222e+07,205.770574,7.580960
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76402,135057,135057,Prescribed Fire,2020,8,8 (3),16.412148,6.641761,Comb_National_Rx_Only_BLM_VTRT_Prescribed_Fire...,Prescribed Fire (3),...,Prescribed fire data in this dataset represent...,None,"Caution, this Prescribed Fire in 2020 overlaps...",0.177425,NaN,No,2168.900740,6.641761e+04,248.914999,1.000000
76403,135058,135058,Prescribed Fire,2020,8,8 (1),7.050837,2.853373,Comb_National_Rx_Only_BLM_VTRT_Prescribed_Fire...,Prescribed Fire (1),...,Prescribed fire data in this dataset represent...,None,"Caution, this Prescribed Fire in 2020 overlaps...",0.374368,NaN,No,978.666221,2.853373e+04,167.983467,1.000000
76404,135059,135059,Prescribed Fire,2020,8,8 (4),9.342668,3.780843,Comb_National_Rx_Only_BLM_VTRT_Prescribed_Fire...,Prescribed Fire (4),...,Prescribed fire data in this dataset represent...,None,"Caution, this Prescribed Fire in 2020 overlaps...",0.123888,NaN,No,1958.326660,3.780843e+04,168.822089,1.000000
76405,135060,135060,Prescribed Fire,2020,8,8 (1),0.996962,0.403456,Comb_National_Rx_Only_BLM_VTRT_Prescribed_Fire...,Prescribed Fire (1),...,Prescribed fire data in this dataset represent...,None,None,0.993809,1.0,No,225.866452,4.034562e+03,655.423767,1.000000


## Compare My Smoke Estimate with US EPA's AQI

### Acquire AQI Data

The first thing we need to do to compare my smoke estimate with the AQI data is to acquire the AQI data. **Note, for many of the subsequent steps, I base my work heavily upon Dr. McDonald's AQI example notebook [14].** 

First, I directly copied a cell from [14] to define useful constants.

In [7]:
#########
#
#    CONSTANTS
#

#
#    This is the root of all AQS API URLs
#
API_REQUEST_URL = 'https://aqs.epa.gov/data/api'

#
#    These are 'actions' we can ask the API to take or requests that we can make of the API
#
#    Sign-up request - generally only performed once - unless you lose your key
API_ACTION_SIGNUP = '/signup?email={email}'
#
#    List actions provide information on API parameter values that are required by some other actions/requests
API_ACTION_LIST_CLASSES = '/list/classes?email={email}&key={key}'
API_ACTION_LIST_PARAMS = '/list/parametersByClass?email={email}&key={key}&pc={pclass}'
API_ACTION_LIST_SITES = '/list/sitesByCounty?email={email}&key={key}&state={state}&county={county}'
#
#    Monitor actions are requests for monitoring stations that meet specific criteria
API_ACTION_MONITORS_COUNTY = '/monitors/byCounty?email={email}&key={key}&param={param}&bdate={begin_date}&edate={end_date}&state={state}&county={county}'
API_ACTION_MONITORS_BOX = '/monitors/byBox?email={email}&key={key}&param={param}&bdate={begin_date}&edate={end_date}&minlat={minlat}&maxlat={maxlat}&minlon={minlon}&maxlon={maxlon}'
#
#    Summary actions are requests for summary data. These are for daily summaries
API_ACTION_DAILY_SUMMARY_COUNTY = '/dailyData/byCounty?email={email}&key={key}&param={param}&bdate={begin_date}&edate={end_date}&state={state}&county={county}'
API_ACTION_DAILY_SUMMARY_BOX = '/dailyData/byBox?email={email}&key={key}&param={param}&bdate={begin_date}&edate={end_date}&minlat={minlat}&maxlat={maxlat}&minlon={minlon}&maxlon={maxlon}'
#
#    It is always nice to be respectful of a free data resource.
#    We're going to observe a 100 requests per minute limit - which is fairly nice
API_LATENCY_ASSUMED = 0.002       # Assuming roughly 2ms latency on the API and network
API_THROTTLE_WAIT = (1.0/100.0)-API_LATENCY_ASSUMED
#
#
#    This is a template that covers most of the parameters for the actions we might take, from the set of actions
#    above. In the examples below, most of the time parameters can either be supplied as individual values to a
#    function - or they can be set in a copy of the template and passed in with the template.
# 
AQS_REQUEST_TEMPLATE = {
    "email":      "",     
    "key":        "",      
    "state":      "",     # the two digit state FIPS # as a string
    "county":     "",     # the three digit county FIPS # as a string
    "begin_date": "",     # the start of a time window in YYYYMMDD format
    "end_date":   "",     # the end of a time window in YYYYMMDD format, begin_date and end_date must be in the same year
    "minlat":    0.0,
    "maxlat":    0.0,
    "minlon":    0.0,
    "maxlon":    0.0,
    "param":     "",     # a list of comma separated 5 digit codes, max 5 codes requested
    "pclass":    ""      # parameter class is only used by the List calls
}



Next, I again reuse a cell from [14] to define a function that requests data from the EPA.

In [8]:
#
#    This implements the list request. There are several versions of the list request that only require email and key.
#    This code sets the default action/requests to list the groups or parameter class descriptors. Having those descriptors 
#    allows one to request the individual (proprietary) 5 digit codes for individual air quality measures by using the
#    param request. Some code in later cells will illustrate those requests.
#
def request_list_info(email_address = None, key = None,
                      endpoint_url = API_REQUEST_URL, 
                      endpoint_action = API_ACTION_LIST_CLASSES, 
                      request_template = AQS_REQUEST_TEMPLATE,
                      headers = None):
    
    #  Make sure we have email and key - at least
    #  This prioritizes the info from the call parameters - not what's already in the template
    if email_address:
        request_template['email'] = email_address
    if key:
        request_template['key'] = key
    
    # For the basic request we need an email address and a key
    if not request_template['email']:
        raise Exception("Must supply an email address to call 'request_list_info()'")
    if not request_template['key']: 
        raise Exception("Must supply a key to call 'request_list_info()'")

    # compose the request
    request_url = endpoint_url+endpoint_action.format(**request_template)
        
    # make the request
    try:
        # Wait first, to make sure we don't exceed a rate limit in the situation where an exception occurs
        # during the request processing - throttling is always a good practice with a free data source
        if API_THROTTLE_WAIT > 0.0:
            time.sleep(API_THROTTLE_WAIT)
        response = requests.get(request_url, headers=headers)
        json_response = response.json()
    except Exception as e:
        print(e)
        json_response = None
    return json_response



According to the professor's example in [14], there are 7 pollutants that sensors may provide data on:
- Carbon Monoxide
- Sulfur dioxide
- Nitrogen dioxide
- Ozone
- "PM10 Total 0-10um STP"
- "PM2.5 - Local Conditions"
- "Acceptable PM2.5 AQI & Speciation Mass"

The natural question is which ones should we focus on? According to a guide on the AQI, Ozone and particle pollution are "the major sources of unhealthy air quality around 99% of the time" ([17] pg 18). Additionally, in an article on why wildfire pollutants are unsafe, the EPA says "Particle pollution represents a main component of wildfire smoke and the principal public health threat" [18]. Based on these reasons, I will ignore the gaseous pollutants: carbon monixide, sulfur dioxide, and nitrogen dioxide for my analysis, to simplify things.

Therefore, reusing and modifying a cell from [14] we define a constant with the codes corresponding to the particle pollutants.

In [9]:
#
#   Given the set of sensor codes, now we can create a parameter list or 'param' value as defined by the AQS API spec.
#   It turns out that we want all of these measures for AQI, but we need to have two different param constants to get
#   all seven of the code types. We can only have a max of 5 sensors/values request per param.
#
#   Gaseous AQI pollutants CO, SO2, NO2, and O2
# AQI_PARAMS_GASEOUS = "42101,42401,42602,44201"
#
#   Particulate AQI pollutants PM10, PM2.5, and Acceptable PM2.5
AQI_PARAMS_PARTICULATES = "81102,88101,88502"
#   
#

**TBD: REDACT!!!!!!!!!!!!**

Now, I followed the steps in [14] to create an account for the API I will be using to get the AQI data. For the sake of privacy, I have redacted the information

In [10]:
USERNAME = "obrienl@uw.edu"
APIKEY = "bolefrog68"

Now we determine which sensors are near Richland WA. Let's start with checking if there are any sensors in the county (Benton). According to [15] and [16], the FIPS code for Benton county is 53005 when you put together the state and county FIPS numbers. Note: I modified the two cells from [14] directly below to use Richland instead of two other city locations.

In [11]:
#
#   We'll use these two city locations in the examples below.
#
CITY_LOCATIONS = {
    'richland' :       {'city'   : 'Richland',
                       'county' : 'Benton',
                       'state'  : 'Washington',
                       'fips'   : '53005',
                       'latlon' : [46.28569070, -119.28446210] }
}


In [12]:
#
#  This list request should give us a list of all the monitoring stations in the county specified by the
#  given city selected from the CITY_LOCATIONS dictionary
#
request_data = AQS_REQUEST_TEMPLATE.copy()
request_data['email'] = USERNAME
request_data['key'] = APIKEY
request_data['state'] = CITY_LOCATIONS['richland']['fips'][:2]   # the first two digits (characters) of FIPS is the state code
request_data['county'] = CITY_LOCATIONS['richland']['fips'][2:]  # the last three digits (characters) of FIPS is the county code

response = request_list_info(request_template=request_data, endpoint_action=API_ACTION_LIST_SITES)

if response["Header"][0]['status'] == "Success":
    print(json.dumps(response['Data'],indent=4))
else:
    print(json.dumps(response,indent=4))


[
    {
        "code": "0001",
        "value_represented": null
    },
    {
        "code": "0002",
        "value_represented": "KENNEWICK - METALINE"
    },
    {
        "code": "0003",
        "value_represented": "Kennewick_S Clodfelter Rd"
    },
    {
        "code": "0004",
        "value_represented": null
    },
    {
        "code": "1001",
        "value_represented": null
    }
]


Looking at the data returned above, I am pleased to see that there are several monitoring stations within Benton County. The professor points out in the assingment instructions that even if there are monitoring stations within the same county as your city, it is unlikely that there will be any within the city limits. Thus, I will also use a "geodetic bounding box" to determine which monitoring stations are even closer to the city (assignment instructions), particularly the closest one. The following cell is copied from [14].

In [13]:
#
#   These are rough estimates for creating bounding boxes based on a city location
#   You can find these rough estimates on the USGS website:
#   https://www.usgs.gov/faqs/how-much-distance-does-a-degree-minute-and-second-cover-your-maps
#
LAT_25MILES = 25.0 * (1.0/69.0)    # This is about 25 miles of latitude in decimal degrees
LON_25MILES = 25.0 * (1.0/54.6)    # This is about 25 miles of longitude in decimal degrees
#
#   Compute a rough estimates for a bounding box around a given place
#   The bounding box is scaled in 50 mile increments. That is the bounding box will have sides that
#   are rough multiples of 50 miles, with the center of the box around the indicated place.
#   The scale parameter determines the scale (size) of the bounding box
#
def bounding_latlon(place=None,scale=1.0):
    minlat = place['latlon'][0] - float(scale) * LAT_25MILES
    maxlat = place['latlon'][0] + float(scale) * LAT_25MILES
    minlon = place['latlon'][1] - float(scale) * LON_25MILES
    maxlon = place['latlon'][1] + float(scale) * LON_25MILES
    return [minlat,maxlat,minlon,maxlon]



After defining the bounding function above, we next resuse a cell from [14] to define a request function.

In [14]:
#
#    This implements the monitors request. This requests monitoring stations. This can be done by state, county, or bounding box. 
#
#    Like the two other functions, this can be called with a mixture of a defined parameter dictionary, or with function
#    parameters. If function parameters are provided, those take precedence over any parameters from the request template.
#
def request_monitors(email_address = None, key = None, param=None,
                          begin_date = None, end_date = None, fips = None,
                          endpoint_url = API_REQUEST_URL, 
                          endpoint_action = API_ACTION_MONITORS_COUNTY, 
                          request_template = AQS_REQUEST_TEMPLATE,
                          headers = None):
    
    #  This prioritizes the info from the call parameters - not what's already in the template
    if email_address:
        request_template['email'] = email_address
    if key:
        request_template['key'] = key
    if param:
        request_template['param'] = param
    if begin_date:
        request_template['begin_date'] = begin_date
    if end_date:
        request_template['end_date'] = end_date
    if fips and len(fips)==5:
        request_template['state'] = fips[:2]
        request_template['county'] = fips[2:]            

    # Make sure there are values that allow us to make a call - these are always required
    if not request_template['email']:
        raise Exception("Must supply an email address to call 'request_monitors()'")
    if not request_template['key']: 
        raise Exception("Must supply a key to call 'request_monitors()'")
    if not request_template['param']: 
        raise Exception("Must supply param values to call 'request_monitors()'")
    if not request_template['begin_date']: 
        raise Exception("Must supply a begin_date to call 'request_monitors()'")
    if not request_template['end_date']: 
        raise Exception("Must supply an end_date to call 'request_monitors()'")
    # Note we're not validating FIPS fields because not all of the monitors actions require the FIPS numbers
    
    # compose the request
    request_url = endpoint_url+endpoint_action.format(**request_template)
    
    # make the request
    try:
        # Wait first, to make sure we don't exceed a rate limit in the situation where an exception occurs
        # during the request processing - throttling is always a good practice with a free data source
        if API_THROTTLE_WAIT > 0.0:
            time.sleep(API_THROTTLE_WAIT)
        response = requests.get(request_url, headers=headers)
        json_response = response.json()
    except Exception as e:
        print(e)
        json_response = None
    return json_response


At this point, we can call the function using a geographic region. I modified a cell from [14] where the professor wrote code to do this. I tested various scale values and found that there is a single monitoring station within the region when the scale is set to $0.25$. It's address as seen below is: 5929 W METALINE (Kennewick Skills Center), Kennewick, WA.

In [15]:
#
#    Create a copy of the AQS_REQUEST_TEMPLATE
#
request_data = AQS_REQUEST_TEMPLATE.copy()
request_data['email'] = USERNAME
request_data['key'] = APIKEY
request_data['param'] = AQI_PARAMS_PARTICULATES     # same particulate request as the one abover
# 
#   Not going to use these - comment them out
#request_data['state'] = CITY_LOCATIONS['bend']['fips'][:2]
#request_data['county'] = CITY_LOCATIONS['bend']['fips'][2:]
#
#   Now, we need bounding box parameters

#------------------------LO--------------------------------
bbox = bounding_latlon(CITY_LOCATIONS['richland'],scale=0.25)

#-------------------------------------------------------------

#   50 mile box
# bbox = bounding_latlon(CITY_LOCATIONS['richland'],scale=1.0)
#   100 mile box
#bbox = bounding_latlon(CITY_LOCATIONS['bend'],scale=2.0)
#   150 mile box
#bbox = bounding_latlon(CITY_LOCATIONS['bend'],scale=3.0)
#   200 mile box
#bbox = bounding_latlon(CITY_LOCATIONS['bend'],scale=4.0)

# the bbox response comes back as a list - [minlat,maxlat,minlon,maxlon]

#   put our bounding box into the request_data
request_data['minlat'] = bbox[0]
request_data['maxlat'] = bbox[1]
request_data['minlon'] = bbox[2]
request_data['maxlon'] = bbox[3]

#
#   we need to change the action for the API from the default to the bounding box - same recent date for now
response = request_monitors(request_template=request_data, begin_date="20210701", end_date="20210731",
                            endpoint_action = API_ACTION_MONITORS_BOX)
#
#
#
if response["Header"][0]['status'] == "Success":
    print(json.dumps(response['Data'],indent=4))
else:
    print(json.dumps(response,indent=4))


[
    {
        "state_code": "53",
        "county_code": "005",
        "site_number": "0002",
        "parameter_code": "81102",
        "poc": 5,
        "parameter_name": "PM10 Total 0-10um STP",
        "open_date": "2019-04-01",
        "close_date": null,
        "concurred_exclusions": null,
        "dominant_source": null,
        "measurement_scale": "NEIGHBORHOOD",
        "measurement_scale_def": "500 M TO 4KM",
        "monitoring_objective": "POPULATION EXPOSURE",
        "last_method_code": "122",
        "last_method_description": "INSTRUMENT MET ONE 4 MODELS - BETA ATTENUATION",
        "last_method_begin_date": "2019-04-01",
        "naaqs_primary_monitor": "Y",
        "qa_primary_monitor": null,
        "monitor_type": "SLAMS",
        "networks": null,
        "monitoring_agency_code": "1136",
        "monitoring_agency": "Washington State Department Of Ecology",
        "si_id": 16457,
        "latitude": 46.21835,
        "longitude": -119.204153,
        "datum

Now, let's expand the search to the sensor locations within 50 miles of Richland to see more results. We again use the code from [14] in the cell below.

In [16]:
#
#    Create a copy of the AQS_REQUEST_TEMPLATE
#
request_data = AQS_REQUEST_TEMPLATE.copy()
request_data['email'] = USERNAME
request_data['key'] = APIKEY
request_data['param'] = AQI_PARAMS_PARTICULATES     # same particulate request as the one abover
# 
#   Not going to use these - comment them out
#request_data['state'] = CITY_LOCATIONS['bend']['fips'][:2]
#request_data['county'] = CITY_LOCATIONS['bend']['fips'][2:]
#
#   Now, we need bounding box parameters

#------------------------LO--------------------------------
#   50 mile box
# bbox = bounding_latlon(CITY_LOCATIONS['richland'],scale=0.25)

#-------------------------------------------------------------

#   50 mile box
bbox = bounding_latlon(CITY_LOCATIONS['richland'],scale=1.0)
#   100 mile box
#bbox = bounding_latlon(CITY_LOCATIONS['bend'],scale=2.0)
#   150 mile box
#bbox = bounding_latlon(CITY_LOCATIONS['bend'],scale=3.0)
#   200 mile box
#bbox = bounding_latlon(CITY_LOCATIONS['bend'],scale=4.0)

# the bbox response comes back as a list - [minlat,maxlat,minlon,maxlon]

#   put our bounding box into the request_data
request_data['minlat'] = bbox[0]
request_data['maxlat'] = bbox[1]
request_data['minlon'] = bbox[2]
request_data['maxlon'] = bbox[3]

#
#   we need to change the action for the API from the default to the bounding box - same recent date for now
response = request_monitors(request_template=request_data, begin_date="20210701", end_date="20210731",
                            endpoint_action = API_ACTION_MONITORS_BOX)
#
#
#
if response["Header"][0]['status'] == "Success":
    print(json.dumps(response['Data'],indent=4))
else:
    print(json.dumps(response,indent=4))


[
    {
        "state_code": "53",
        "county_code": "005",
        "site_number": "0002",
        "parameter_code": "81102",
        "poc": 5,
        "parameter_name": "PM10 Total 0-10um STP",
        "open_date": "2019-04-01",
        "close_date": null,
        "concurred_exclusions": null,
        "dominant_source": null,
        "measurement_scale": "NEIGHBORHOOD",
        "measurement_scale_def": "500 M TO 4KM",
        "monitoring_objective": "POPULATION EXPOSURE",
        "last_method_code": "122",
        "last_method_description": "INSTRUMENT MET ONE 4 MODELS - BETA ATTENUATION",
        "last_method_begin_date": "2019-04-01",
        "naaqs_primary_monitor": "Y",
        "qa_primary_monitor": null,
        "monitor_type": "SLAMS",
        "networks": null,
        "monitoring_agency_code": "1136",
        "monitoring_agency": "Washington State Department Of Ecology",
        "si_id": 16457,
        "latitude": 46.21835,
        "longitude": -119.204153,
        "datum

**TBD Cleanup text below!!!!!**

**how do I use this result? Should I focus on a single monitor or combine results??????**

What do I know:
- The monitor stations in the same county
- The closest monitor station and if I do the math, it's distance from Richland
- There are 3 types of particulate a sensor can measure. 
- There are three sites with 50 miles of Richland. Unfortuneately, the earliest was not established until 2003. 

By running the cell above, we see the sensors appearing within 50 miles of Richland and the pollutants that they provide. I summarized the results as follows:
- Site 0002: PM10 Total 0-10um STP, open_date 2019-04-01 --> 5929 W METALINE, Kennewick
- Site 0006: PM10 Total 0-10um STP, open_date 2019-10-03 --> 755 MAPLE STREET, Burbank
- Site 0002: Acceptable PM2.5 AQI & Speciation Mass, open_date 2003-01-15 --> 200 PEPIOT WAY, Mesa
- Site 0002: Acceptable PM2.5 AQI & Speciation Mass, open_date 2005-10-19 --> 5929 W METALINE, Kenewick

I do not know the definitions of several of the terms provided in the results. In particular, I assume that the site number is a unique identifier for each location, however, this is contradicted by the fact that Site 0002 is linked to a site in Kennewick and a site in Mesa. Regardless, looking at the types of pollutant info provided, we see the two kinds are "PM10 Total 0-10um STP" and "Acceptable PM2.5 AQI & Speciation Mass". Since both of these are provided by the Kennewick 0002 site, I will solely reference that site for the AQI for my analysis, as it is the closest one to Richland - however, I do note that I am sacrificing 2 years of PM2.5 data as the Kennewick location started providing this over two years later than the Mesa 0006 site.

Armed with the knowledge of which sensor we will use and which pollutant types it provides data on, we proceed by writing a new API request function to get the daily values of AQI. The code in the cell below resuses a cell from [14] verbatim.

In [17]:
#
#    This implements the daily summary request. Daily summary provides a daily summary value for each sensor being requested
#    from the start date to the end date. 
#
#    Like the two other functions, this can be called with a mixture of a defined parameter dictionary, or with function
#    parameters. If function parameters are provided, those take precedence over any parameters from the request template.
#
def request_daily_summary(email_address = None, key = None, param=None,
                          begin_date = None, end_date = None, fips = None,
                          endpoint_url = API_REQUEST_URL, 
                          endpoint_action = API_ACTION_DAILY_SUMMARY_COUNTY, 
                          request_template = AQS_REQUEST_TEMPLATE,
                          headers = None):
    
    #  This prioritizes the info from the call parameters - not what's already in the template
    if email_address:
        request_template['email'] = email_address
    if key:
        request_template['key'] = key
    if param:
        request_template['param'] = param
    if begin_date:
        request_template['begin_date'] = begin_date
    if end_date:
        request_template['end_date'] = end_date
    if fips and len(fips)==5:
        request_template['state'] = fips[:2]
        request_template['county'] = fips[2:]            

    # Make sure there are values that allow us to make a call - these are always required
    if not request_template['email']:
        raise Exception("Must supply an email address to call 'request_daily_summary()'")
    if not request_template['key']: 
        raise Exception("Must supply a key to call 'request_daily_summary()'")
    if not request_template['param']: 
        raise Exception("Must supply param values to call 'request_daily_summary()'")
    if not request_template['begin_date']: 
        raise Exception("Must supply a begin_date to call 'request_daily_summary()'")
    if not request_template['end_date']: 
        raise Exception("Must supply an end_date to call 'request_daily_summary()'")
    # Note we're not validating FIPS fields because not all of the daily summary actions require the FIPS numbers
        
    # compose the request
    request_url = endpoint_url+endpoint_action.format(**request_template)
        
    # make the request
    try:
        # Wait first, to make sure we don't exceed a rate limit in the situation where an exception occurs
        # during the request processing - throttling is always a good practice with a free data source
        if API_THROTTLE_WAIT > 0.0:
            time.sleep(API_THROTTLE_WAIT)
        response = requests.get(request_url, headers=headers)
        json_response = response.json()
    except Exception as e:
        print(e)
        json_response = None
    return json_response



Next, we tidy up the output by defining a summary function. The code cell below is directly copied from [14].

In [18]:
#
#    This is a list of field names - data - that will be extracted from each record
#
EXTRACTION_FIELDS = ['sample_duration','observation_count','arithmetic_mean','aqi']

#
#    The function creates a summary record
def extract_summary_from_response(r=None, fields=EXTRACTION_FIELDS):
    ## the result will be structured around monitoring site, parameter, and then date
    result = dict()
    data = r["Data"]
    for record in data:
        # make sure the record is set up
        site = record['site_number']
        param = record['parameter_code']
        #date = record['date_local']    # this version keeps the respnse value YYYY-
        date = record['date_local'].replace('-','') # this puts it in YYYYMMDD format
        if site not in result:
            result[site] = dict()
            result[site]['local_site_name'] = record['local_site_name']
            result[site]['site_address'] = record['site_address']
            result[site]['state'] = record['state']
            result[site]['county'] = record['county']
            result[site]['city'] = record['city']
            result[site]['pollutant_type'] = dict()
        if param not in result[site]['pollutant_type']:
            result[site]['pollutant_type'][param] = dict()
            result[site]['pollutant_type'][param]['parameter_name'] = record['parameter']
            result[site]['pollutant_type'][param]['units_of_measure'] = record['units_of_measure']
            result[site]['pollutant_type'][param]['method'] = record['method']
            result[site]['pollutant_type'][param]['data'] = dict()
        if date not in result[site]['pollutant_type'][param]['data']:
            result[site]['pollutant_type'][param]['data'][date] = list()
        
        # now extract the specified fields
        extract = dict()
        for k in fields:
            if str(k) in record:
                extract[str(k)] = record[k]
            else:
                # this makes sure we always have the requested fields, even if
                # we have a missing value for a given day/month
                extract[str(k)] = None
        
        # add this extraction to the list for the day
        result[site]['pollutant_type'][param]['data'][date].append(extract)
    
    return result


Next we start the proecess for requesting the daily summary data. The cell below is reused and modified from [14]. Some of the main changes I made to the cell are that:
- I implemented the ability to loop through multiple years at once
- I removed the request for gaseous AQI
- I added the function call to extract_summary_from_response()
- Commented out passing state and county arguments to try to operate by bounding box instead to get a specific site
- Added code to pass the latlongs for the request template
- Changed the endpoint_action to use a geographic box instead of a county.
- TBD: pass the 4 latlong parameters to request_data
- TBD: update particulate codes?

**TBD Should I update which particulates I am grabbing?**

In [19]:
#   Particulate AQI pollutants PM10, PM2.5, and Acceptable PM2.5
AQI_PARAMS_PARTICULATES = "81102,88101,88502"

In [20]:
from datetime import date 

request_data = AQS_REQUEST_TEMPLATE.copy()
request_data['email'] = USERNAME
request_data['key'] = APIKEY
request_data['param'] = AQI_PARAMS_PARTICULATES
# request_data['state'] = CITY_LOCATIONS['richland']['fips'][:2]
# request_data['county'] = CITY_LOCATIONS['richland']['fips'][2:]

bbox = bounding_latlon(CITY_LOCATIONS['richland'],scale=0.25)

#   put our bounding box into the request_data
request_data['minlat'] = bbox[0]
request_data['maxlat'] = bbox[1]
request_data['minlon'] = bbox[2]
request_data['maxlon'] = bbox[3]


# request daily summary data for all years
# START_DATE = '20220101' #should be 1963
# PRESENT_DATE = '20231108'

START_DATE = '19630101' #should be 1963
PRESENT_DATE = '20231108'

# get every year between both dates
# I learned to do this from: https://stackoverflow.com/questions/23590042/get-all-years-between-two-dates-in-python
start_date = datetime.strptime(START_DATE, '%Y%m%d')
end_date = datetime.strptime(PRESENT_DATE, '%Y%m%d')


# Retrieve the AQI data for each year
particulate_aqi_ls = []
success_years = []

for year in range(start_date.year, end_date.year + 1):
    
    # determine current start and end dates
    # I referenced: https://www.geeksforgeeks.org/add-years-to-datetime-object-in-python/
    # and: https://stackoverflow.com/questions/15741618/add-one-year-in-current-date-python
    current_start_date = date(year, 1, 1)
    
    if year == end_date.year: # if you've reached final year you won't grab entire year
        current_end_date = end_date
    
    else: # otherwise your current end date is the last day of December of current year
        current_end_date = date(year, 12, 31)
    
    # get the dates in string format
    # referenced: https://www.programiz.com/python-programming/datetime/strftime
    current_start_date_str = current_start_date.strftime('%Y%m%d')
    current_end_date_str = current_end_date.strftime('%Y%m%d')
    
    # call the api -------------------------------------------------------------------------------------------------------------
    particulate_aqi = request_daily_summary(request_template=request_data, 
                                            begin_date=current_start_date_str, end_date=current_end_date_str, 
                                            endpoint_action = API_ACTION_DAILY_SUMMARY_BOX)
    # check that it worked
    print("Response for the particulate pollutants in {}...".format(year))
    #
    if particulate_aqi["Header"][0]['status'] == "Success":
    #     print(json.dumps(particulate_aqi['Data'],indent=4))
        print('Success')
        success_years.append(str(year))
        
        # tidy up output
        extract_particulate_dict = extract_summary_from_response(particulate_aqi)

        #store data
        particulate_aqi_ls.append(extract_particulate_dict)
        
    elif particulate_aqi["Header"][0]['status'].startswith("No data "):
        print("Looks like the response generated no data. You might take a closer look at your request and the response data.")
    else:
        print(json.dumps(particulate_aqi,indent=4))
    #     print('No idea what is going on')

Response for the particulate pollutants in 1963...
Looks like the response generated no data. You might take a closer look at your request and the response data.
Response for the particulate pollutants in 1964...
Looks like the response generated no data. You might take a closer look at your request and the response data.
Response for the particulate pollutants in 1965...
Looks like the response generated no data. You might take a closer look at your request and the response data.
Response for the particulate pollutants in 1966...
Looks like the response generated no data. You might take a closer look at your request and the response data.
Response for the particulate pollutants in 1967...
Looks like the response generated no data. You might take a closer look at your request and the response data.
Response for the particulate pollutants in 1968...
Looks like the response generated no data. You might take a closer look at your request and the response data.
Response for the particulate

Call the extract summary function to tidy up the results. The cell below is reused and modified from [14].

In [21]:
# extract_particulate = extract_summary_from_response(particulate_aqi)
# print("Summary of particulate extraction ...")
# print(json.dumps(extract_particulate,indent=4))

Let's recap where we are at at this point. We are seeking to retrieve a value for AQI from the EPA API that we can compare to our smoke estimates. We have decided based on our exploration to get data from the Site 0002 in Kennewick Washington, another town in Benton county. Additionally, we have demonstrated that we can get the daily summary data for this site for a specified date range. 

Now, since we are ultimately going to chart the smoke estimate and AQI at the annual level, we need to figure out a way to move from a daily granularity AQI for two provided particle types, to a single AQI value for each year.

So, first we store the output of the extract summary as a data type that we can work with.

In [22]:
# extract_particulate_dict = extract_summary_from_response(particulate_aqi)

Now, let's retrive the information we need and make this easier to work with by storing the aqi in a couple dictionaries. 

First, let's take a look at how many sites there are for each year

In [23]:
year_list = success_years
year_index = 0

# iterate through all years of data
for extract_particulate_dict in particulate_aqi_ls: # for each year
    year = year_list[year_index]
    print("------------ Retrieving data for {} ------------".format(year))
        
    # iterate through all sites
    site_keys = extract_particulate_dict.keys()
    print("There are {} sites".format(len(site_keys)))
    for site_key in site_keys:
        print(site_key)
        
    year_index = year_index + 1

------------ Retrieving data for 1985 ------------
There are 1 sites
0001
------------ Retrieving data for 1986 ------------
There are 1 sites
0001
------------ Retrieving data for 1987 ------------
There are 1 sites
0001
------------ Retrieving data for 1988 ------------
There are 1 sites
0001
------------ Retrieving data for 1989 ------------
There are 1 sites
0001
------------ Retrieving data for 1990 ------------
There are 1 sites
0001
------------ Retrieving data for 1991 ------------
There are 1 sites
0001
------------ Retrieving data for 1992 ------------
There are 1 sites
0001
------------ Retrieving data for 1993 ------------
There are 1 sites
0001
------------ Retrieving data for 1994 ------------
There are 2 sites
0001
0002
------------ Retrieving data for 1995 ------------
There are 1 sites
0002
------------ Retrieving data for 1996 ------------
There are 1 sites
0002
------------ Retrieving data for 1997 ------------
There are 1 sites
0002
------------ Retrieving data for 

By inspecting the output above, we see that for every year except 1994, there is a single site. Additionally, from 1985 to 1994 site 0001 is used and from 1994 to the present day site 0002 is used. Now, to make the coding task easier, I am going to ignore site 0001 for 1994.

**TBD Decision, right now, the next cell allows AQI to be None if necessary. should I change to a constant like '-1'?**

In [24]:
# Reference:https://www.geeksforgeeks.org/range-to-a-list-in-python/
year_list = success_years
year_index = 0

# define my dictionaries outside the loop so that the hold values for all years
pm10_aqi_dict = {}
pm25_aqi_dict = {}

pm10_is_available = False
pm25_is_available = False

# iterate through all years of data
for extract_particulate_dict in tqdm(particulate_aqi_ls): # for each year
    year = year_list[year_index]
    print("------------ Retrieving data for {} ------------".format(year))
    
    # select the site data:
    year_num = int(year)
    
    # use the hueristic described above for knowing which site to pick
    if year_num < 1994:
        site_data = extract_particulate_dict['0001']
    
    else:
        site_data = extract_particulate_dict['0002']
    
    # get the data for each pollutant type
    try: #try to get pm10 data
        pm10_data_dict = site_data['pollutant_type']['81102']['data']
    except:
        print("No pm10 data")
        pm10_data_dict = None
    else:
        # if it worked set the value to True
        pm10_is_available = True
        
    try: #try to get the pm2.5 data
        acceptable_pm2_5_data_dict = site_data['pollutant_type']['88502']['data']
    except:
        print("no pm2.5 data")
        acceptable_pm2_5_data_dict = None
    else:
        # if it worked set the value to True
        pm25_is_available = True

    # check if don't have either pollutant data
    if (not pm10_is_available) and (not pm25_is_available):
        print('No pollutant data !!!!!!!!!!!!!!!!!!!!!!!!!!')
        continue
    
    else:
        pass
    
    # iterate over the dictionaries
    # I viewed https://www.geeksforgeeks.org/iterate-over-a-dictionary-in-python/
    # to learn how to iterate over a dictionary

    if (pm10_is_available):
        # first we get lists of the keys
        pm10_data_keys = pm10_data_dict.keys()
   
        # iterate through the dictionary of dates for pm10 and grab the aqi for each
        print("Retrieving pm10 AQI data")
        pm10_missing_count = 0
        pm10_null_count = 0
        for key in pm10_data_keys:

            try:
                aqi = pm10_data_dict[key][1]['aqi']

            except:
                aqi = None
                pm10_missing_count = pm10_missing_count + 1
                print("AQI for pm10 data with date {} is missing".format(key))

            #flag potential issues
            else:
                if aqi is None:
                    print("AQI for pm10 data with date {} is null".format(key))
                    pm10_null_count = pm10_null_count + 1

            pm10_aqi_dict[key] = aqi

        print("There were {} instances of missing data.".format(pm10_missing_count)) #where no 24 hr data
        print("There were {} null values in the data.".format(pm10_null_count))
        print("Total null values retrived: {}".format(pm10_missing_count + pm10_null_count))

    if (pm25_is_available):
        # first we get lists of the keys
        acceptable_pm2_5_data_keys = acceptable_pm2_5_data_dict.keys()

        # iterate through the dictionary of dates for pm2.5 acceptability and grab the aqi for each
        print('')
        print("Retrieving pm2.5 Acceptable AQI data")
        pm25_missing_count = 0
        pm25_nulls_count = 0
        for key in acceptable_pm2_5_data_keys:

            try:
                aqi = acceptable_pm2_5_data_dict[key][1]['aqi']

            except:
                aqi = None
                pm25_missing_count = pm25_missing_count + 1
                print("AQI for pm2.5 acceptable data with date {} is null".format(key))

            #flag potential issues
            else:
                if aqi is None:
                    print("AQI for pm2.5 acceptable data with date {} is null".format(key))
                    pm25_nulls_count = pm25_nulls_count + 1

            pm25_aqi_dict[key] = aqi

        print("There were {} instances of missing data.".format(pm25_missing_count)) #where no 24 hr data
        print("There were {} null values in the data.".format(pm25_nulls_count))
        print("Total null values retrived: {}".format(pm25_missing_count + pm25_nulls_count))

    #update year index
    year_index = year_index + 1

100%|█████████████████████████████████████████████████████████████████████████████████| 39/39 [00:00<00:00, 3010.43it/s]

------------ Retrieving data for 1985 ------------
no pm2.5 data
Retrieving pm10 AQI data
AQI for pm10 data with date 19851009 is missing
AQI for pm10 data with date 19851010 is missing
AQI for pm10 data with date 19851012 is missing
AQI for pm10 data with date 19851014 is missing
AQI for pm10 data with date 19851016 is missing
AQI for pm10 data with date 19851018 is missing
AQI for pm10 data with date 19851020 is missing
AQI for pm10 data with date 19851022 is missing
AQI for pm10 data with date 19851024 is missing
AQI for pm10 data with date 19851026 is missing
AQI for pm10 data with date 19851028 is missing
AQI for pm10 data with date 19851030 is missing
AQI for pm10 data with date 19851101 is missing
AQI for pm10 data with date 19851103 is missing
AQI for pm10 data with date 19851106 is missing
AQI for pm10 data with date 19851109 is missing
AQI for pm10 data with date 19851111 is missing
AQI for pm10 data with date 19851113 is missing
AQI for pm10 data with date 19851115 is missin

In [25]:
asdf

NameError: name 'asdf' is not defined

**TBD Delete:**
Notes:
- why is there no pm2.5 data for 2003? Wait. This is only showing when the data was missing. Nevermind

In [26]:
pm10_aqi_dict

{'19851009': None,
 '19851010': None,
 '19851012': None,
 '19851014': None,
 '19851016': None,
 '19851018': None,
 '19851020': None,
 '19851022': None,
 '19851024': None,
 '19851026': None,
 '19851028': None,
 '19851030': None,
 '19851101': None,
 '19851103': None,
 '19851106': None,
 '19851109': None,
 '19851111': None,
 '19851113': None,
 '19851115': None,
 '19851117': None,
 '19851119': None,
 '19851121': None,
 '19851123': None,
 '19851129': None,
 '19851201': None,
 '19851205': None,
 '19851207': None,
 '19851209': None,
 '19851211': None,
 '19851213': None,
 '19851215': None,
 '19851220': None,
 '19851222': None,
 '19851224': None,
 '19851226': None,
 '19851228': None,
 '19851230': None,
 '19860101': None,
 '19860103': None,
 '19860105': None,
 '19860109': None,
 '19860111': None,
 '19860113': None,
 '19860115': None,
 '19860117': None,
 '19860119': None,
 '19860121': None,
 '19860123': None,
 '19860125': None,
 '19860127': None,
 '19860129': None,
 '19860131': None,
 '19860202':

In [27]:
len(pm10_aqi_dict.keys())

11491

In [28]:
pm25_aqi_dict

{'20010405': 6,
 '20010406': 3,
 '20010407': 7,
 '20010408': 15,
 '20010409': 18,
 '20010410': 10,
 '20010411': 15,
 '20010412': 15,
 '20010413': 13,
 '20010414': 17,
 '20010415': 25,
 '20010416': 50,
 '20010417': 27,
 '20010418': 25,
 '20010419': 28,
 '20010420': 28,
 '20010421': 19,
 '20010422': 25,
 '20010423': 19,
 '20010424': 28,
 '20010425': 49,
 '20010426': 41,
 '20010427': 25,
 '20010428': 22,
 '20010429': 15,
 '20010430': 16,
 '20010501': None,
 '20010502': 13,
 '20010503': 18,
 '20010504': 31,
 '20010505': 20,
 '20010506': 27,
 '20010507': 38,
 '20010508': 39,
 '20010509': 31,
 '20010510': 36,
 '20010511': 48,
 '20010512': 57,
 '20010513': 21,
 '20010514': 21,
 '20010515': 11,
 '20010516': 20,
 '20010517': 22,
 '20010518': 22,
 '20010519': 66,
 '20010520': 10,
 '20010521': 26,
 '20010522': 33,
 '20010523': 53,
 '20010524': 43,
 '20010525': 32,
 '20010526': 44,
 '20010527': 30,
 '20010528': 31,
 '20010529': 24,
 '20010530': 23,
 '20010531': 17,
 '20010601': 29,
 '20010602': 18

In [29]:
len(pm25_aqi_dict.keys())

7433

Now, my classmate, Emily Rolen, recommended I select the larger AQI value of pm10 and pm2.5 for my overall AQI value for each day, and I like this idea. Based upon my exploratory work earlier into the 0002 Site sensor, we know that the pm10 detector has an open date of 2019-04-01, while the pm2.5 acceptable has an open date of 2005-10-19. Based upon this info, let's find the max AQI and store it.

**TBD Decision, right now, the next cell allows AQI to be None if necessary. should I change to a constant like '-1'?**
**Also, issue of missing keys from pm10!!!??????????????**

**TBD: rewrite the code below. Instead iterate through every day in the range. Check if the key exists in both dictionaries and compare values. This will handle coverage issue.**

In [ ]:
pm10_aqi_keys = pm10_aqi_dict.keys()
pm25_aqi_keys = pm25_aqi_dict.keys()

aqi_dict = {}
pm25_key_index = 0
# iterate through the pm2.5 data. we can assume the pm2.5 aqi dataframe has more values since it's monitor has been open longer
for key in tqdm(pm25_aqi_keys):
    pm25_aqi = pm25_aqi_dict[key]
    
    #check if we have reached first possible date for both dataframes to have values
    try:
        # check if the pm10 dictionary has an entry for the current date/key
        pm10_aqi = pm10_aqi_dict[key]
    
    except: #if key doesn't exist use the value from pm25
        max_aqi = pm25_aqi
    
    else: #if pm10 does have an entry for that key choose the max
        
        # handle null values
        if (pm10_aqi is None) and (pm25_aqi is None):
            max_aqi = None
            
        elif pm10_aqi is None:
            max_aqi = pm25_aqi
            
        elif pm25_aqi is None:
            max_aqi = pm10_aqi
            
        else: # if neither are null compare them and pick largest
            if pm10_aqi >= pm25_aqi:
                max_aqi = pm10_aqi
            else:
                max_aqi = pm25_aqi 
        
    #save the value
    aqi_dict[key] = max_aqi

In [ ]:
# aqi_dict

**TBD: understand why comment doesn't work**

Now we convert the AQI dictionary to a pandas data frame

In [ ]:
# I referenced: https://www.geeksforgeeks.org/how-to-create-dataframe-from-dictionary-in-python-pandas/#
# and: https://stackoverflow.com/questions/17839973/constructing-pandas-dataframe-from-values-in-variables-gives-valueerror-if-usi
# and: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html
# and: https://sparkbyexamples.com/pandas/pandas-create-new-dataframe-by-selecting-specific-columns/#:~:text=You%20can%20create%20a%20new,added%20to%20the%20original%20ones.
# df_aqi = pd.DataFrame.from_records(aqi_dict)
# helpful to understand erorr in line above: df_aqi = pd.DataFrame().assign(aqi_dict)

df_daily_aqi = pd.DataFrame({
    'Date' : aqi_dict.keys(),
    'AQI' : aqi_dict.values()
})
df_daily_aqi

**TBD:**

Potential Issues/Big Assumptions:
- This method will ensure that every date in the pm2.5 data will be included, but no check for whether every date in pm10 will be.

Next, we find the annual AQI. First we have to add a column to the daily aqi data frame representing each year

In [ ]:
# Sources I referenced to review how to format dates:
# https://docs.python.org/3/library/datetime.html#format-codes
# https://stackoverflow.com/questions/55542280/why-does-python-3-find-this-iso8601-date-2019-04-05t165526z-invalid
# https://stackoverflow.com/questions/19934248/nameerror-name-datetime-is-not-defined

my_date = datetime.strptime('20210701', '%Y%m%d')
my_date.year

year_list = []
# iterate through the dataframe
for index, row in df_daily_aqi.iterrows():
    date = row['Date']
    my_date = datetime.strptime(date, '%Y%m%d')
    year_list.append(my_date.year)
    
# add the years to the dataframe
df_daily_aqi['Year'] = year_list
df_daily_aqi = df_daily_aqi[['Year', 'Date', 'AQI']]
df_daily_aqi

Note, that there may be days when the sensor did not record data for either pollutant type. For example, I tested the code on acquiring data from 2021 and the dataframe with daily AQI values had data for 343 rows, not 365. While this is not ideal, we proceed using this method as it still provides some helpful data.

**LO: If have time add a column daily aqi dataframe for tracking what proportion of days each year had data.**

Now that we have the daily aqi, let's find the average aqi for the recorded days for each year and save the result.

In [ ]:
# I viewed: https://pandas.pydata.org/docs/reference/api/pandas.core.groupby.DataFrameGroupBy.aggregate.html
# to remind myself how to do a groupby
df_annual_avg_aqi = df_daily_aqi.groupby('Year').agg('mean')
df_annual_avg_aqi

In [ ]:
df_annual_avg_aqi.to_csv('Annual_AQI_Average.csv')

Steps:
- create a dataframe
- add a column for smoke estimate calculation

For the step 2 visualizations, the data should be in a dataframe form. That way I can group by year



## References:
- Learning to use .gitignore:
    * https://stackoverflow.com/questions/19663093/apply-gitignore-on-an-existing-repository-already-tracking-large-number-of-file
    * https://stackoverflow.com/questions/40441450/relative-parent-directory-path-for-gitignore
- Learning to work with GIS data:
    * [3] wildfire_geo_proximity_example.ipynb. //"This code example was developed by Dr. David W. McDonald for use in DATA 512, a course in the UW MS Data Science degree program. This code is provided under the [Creative Commons](https://creativecommons.org) [CC-BY license](https://creativecommons.org/licenses/by/4.0/). Revision 1.0 - August 13, 2023"
- [4]. Proffessor's wildfire module
- [5]. https://www.latlong.net/place/west-richland-wa-usa-25340.html
- [6]. https://stackoverflow.com/questions/12268930/create-json-with-multiple-dictionaries-python
- [7]. https://www.geeksforgeeks.org/reading-and-writing-json-to-a-file-in-python/#
- [8]. https://www.w3schools.com/python/python_try_except.asp
- [9]. 'DATA 512 Homework 1 Acquire Process and Analyze Data.ipynb' at: https://github.com/logan-obrien/data-512-homework_1.git
- [10]. https://www.airnow.gov/aqi/aqi-basics/
- [11]. https://www.google.com/search?q=how+to+convert+acres+to+square+miles&rlz=1C1RXQR_enUS1019US1019&oq=how+to+convert+from+acres+to+s&gs_lcrp=EgZjaHJvbWUqCAgDEAAYFhgeMgYIABBFGDkyCAgBEAAYFhgeMggIAhAAGBYYHjIICAMQABgWGB4yCAgEEAAYFhgeMgoIBRAAGIYDGIoFMgoIBhAAGIYDGIoF0gEIOTU3MmowajeoAgCwAgA&sourceid=chrome&ie=UTF-8
- [12]. https://www.geeksforgeeks.org/numpy-argmax-python/
- [13]. https://numpy.org/doc/stable/reference/generated/numpy.argmax.html
- [14]. epa_air_quality_history_example.ipynb //"This code example was developed by Dr. David W. McDonald for use in DATA 512, a course in the UW MS Data Science degree program. This code is provided under the [Creative Commons](https://creativecommons.org) [CC-BY license](https://creativecommons.org/licenses/by/4.0/). Revision 1.1 - September 5, 2023"
- [15]. https://www.census.gov/library/reference/code-lists/ansi.html#cousub 
- [16]. https://www2.census.gov/geo/docs/reference/codes2020/cousub/st53_wa_cousub2020.txt
- [17]. https://www.airnow.gov/sites/default/files/2020-05/aqi-technical-assistance-document-sept2018.pdf
- [18]. https://www.epa.gov/wildfire-smoke-course/why-wildfire-smoke-health-concern